In [59]:
import pandas as pd
import tensorflow as tf
import numpy as np
import scipy.io
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import metrics
from keras.layers.core import Dense, Activation
import pandas
import keras

In [60]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","outcome"]

df = pd.read_csv("kddcup99.csv", header=None, names = col_names)

In [61]:
#fill in NAN values 
#fill forward, fills in NaN values with the last corressponding value before 
#NaN Value has been reached

df = df.fillna(method='ffill')

df[0:5]

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9.0,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19.0,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29.0,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39.0,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49.0,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [62]:
df.groupby('outcome')['outcome'].count()

outcome
back.                 2103
buffer_overflow.        12
ftp_write.               8
guess_passwd.           53
imap.                   12
ipsweep.               862
land.                   17
loadmodule.              8
multihop.                7
neptune.             41122
nmap.                  231
normal.              71225
perl.                    2
phf.                     3
pod.                   102
portsweep.             639
rootkit.                 7
satan.                1587
smurf.              197151
spy.                     2
teardrop.              397
warezclient.          1020
warezmaster.            20
Name: outcome, dtype: int64

In [63]:
#split into features and outcomes

x = df.drop(['outcome'], axis = 1 )
y = df[['outcome']]

In [64]:
x[0:5]

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9.0,9.0,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,tcp,http,SF,239,486,0,0,0,0,...,19.0,19.0,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29.0,29.0,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39.0,39.0,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49.0,49.0,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [65]:
y[0:5]

,outcome
0,normal.
1,normal.
2,normal.
3,normal.
4,normal.


In [66]:
dummies_1= pd.get_dummies(x['protocol_type'])#
dummies_2 = pd.get_dummies(x['service'])
dummies_3 = pd.get_dummies(x['flag'])

#x = x.drop(['protocol_type','service','flag'], axis = 1)

#x = x.join(x)

In [67]:
merged = pandas.concat([x, dummies_1], axis = 'columns')

merged[0:5]

#dummy variable trap 
merged = merged.drop(['icmp'], axis = 1)

In [68]:
merged = pandas.concat([x, dummies_2], axis = 'columns')

merged[0:5]

#dummy variable trap 

merged = merged.drop(['whois'], axis = 1)

In [69]:
merged = pandas.concat([x, dummies_3], axis = 'columns')

merged[0:5]

merged = merged.drop(['SH'], axis = 1)

In [70]:
x = merged

x = x.drop(['protocol_type','service','flag'], axis = 1)

x[0:5]

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,OTH,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [71]:
#turn malicious packets into true and normal into false

y['Target'] = y['outcome'] != 'normal.'
y = y.drop('outcome', axis=1)

C:\Users\avmas\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:


# Turn Outcomes into dummy variables 

dummies = pd.get_dummies(y['Target'])

# This will turn BENIGN values into 1 being False and all malicious packets into 0 being True. 
y = dummies.values

print(y)

[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [0 1]]


In [76]:
# Get column names first
names = x.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit data on the scaler object
scaled_df = scaler.fit_transform(x)
new_scaled_df = pd.DataFrame(scaled_df, columns=names)

x = pd.DataFrame(new_scaled_df)

x[0:5]

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,OTH,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF
0,-0.033283,-0.003424,0.105603,-0.007328,-0.037816,-0.003212,-0.052508,-0.012108,2.017239,-0.007657,...,-0.003078,-0.12043,-0.018473,-0.00562,-0.046905,-0.387647,-0.012055,-0.007328,-0.005332,0.416913
1,-0.033283,-0.003377,-0.017420,-0.007328,-0.037816,-0.003212,-0.052508,-0.012108,2.017239,-0.007657,...,-0.003078,-0.12043,-0.018473,-0.00562,-0.046905,-0.387647,-0.012055,-0.007328,-0.005332,0.416913
2,-0.033283,-0.003380,0.003670,-0.007328,-0.037816,-0.003212,-0.052508,-0.012108,2.017239,-0.007657,...,-0.003078,-0.12043,-0.018473,-0.00562,-0.046905,-0.387647,-0.012055,-0.007328,-0.005332,0.416913
3,-0.033283,-0.003393,0.003670,-0.007328,-0.037816,-0.003212,-0.052508,-0.012108,2.017239,-0.007657,...,-0.003078,-0.12043,-0.018473,-0.00562,-0.046905,-0.387647,-0.012055,-0.007328,-0.005332,0.416913
4,-0.033283,-0.003394,0.020894,-0.007328,-0.037816,-0.003212,-0.052508,-0.012108,2.017239,-0.007657,...,-0.003078,-0.12043,-0.018473,-0.00562,-0.046905,-0.387647,-0.012055,-0.007328,-0.005332,0.416913


In [77]:
#Perform PCA to reduce dimensionality of feature vector for better accuracy
#reduce from 37 to 2 dimensional space

pca=PCA(n_components = 2)
pca.fit(x)
x_scaled = pca.transform(x)
print ("Original Shape:   ", x.shape)
print ("Transformed Shape:",x_scaled.shape)

x = x_scaled

print(x)

Original Shape:    (316590, 48)
Transformed Shape: (316590, 2)
[[ 0.36174106  0.937596  ]
 [ 0.35649047  0.91992472]
 [ 0.33224903  0.89852224]
 ...
 [-1.66642125 -0.41085014]
 [-1.66642125 -0.41085014]
 [-1.66642125 -0.41085014]]


In [128]:
#Split into test and training sets. 25% test sample 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.40, random_state=0)



In [129]:
# Create Neural Network, with 3 hidden layers and 1 output layer being subjected to the softmax probability function

model = Sequential()

model.add(Dense(60, input_dim = x.shape[1], activation='relu', kernel_initializer='random_normal'))
model.add(Dense(30, input_dim = x.shape[1], activation='relu', kernel_initializer='random_normal', )) 
model.add(Dense(20, input_dim = x.shape[1],  activation='relu', kernel_initializer='random_normal',)) 
model.add(Dense(2, input_dim = x.shape[1],  activation='relu', kernel_initializer='random_normal',)) 
model.add(Dense(y.shape[1],activation='softmax')) #takes in an input and spits out 1D vector

#Add in hyper parameters
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
#Early stopping used to stop model from over or under fitting - stops when accuracy is no longer improving 
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=6, verbose=3, mode='auto', restore_best_weights=False, baseline=None)
#Fit the model
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=1,epochs=100)

Train on 189954 samples, validate on 126636 samples
Epoch 1/100
189954/189954 [==============================] - 14s 73us/step - loss: 0.0899 - acc: 0.9716 - val_loss: 0.0587 - val_acc: 0.9838
Epoch 2/100
189954/189954 [==============================] - 13s 66us/step - loss: 0.0574 - acc: 0.9837 - val_loss: 0.0553 - val_acc: 0.9841
Epoch 3/100
189954/189954 [==============================] - 13s 68us/step - loss: 0.0473 - acc: 0.9849 - val_loss: 0.0401 - val_acc: 0.9861
Epoch 4/100
189954/189954 [==============================] - 14s 74us/step - loss: 0.0357 - acc: 0.9867 - val_loss: 0.0364 - val_acc: 0.9866
Epoch 5/100
189954/189954 [==============================] - 14s 74us/step - loss: 0.0323 - acc: 0.9873 - val_loss: 0.0328 - val_acc: 0.9868
Epoch 6/100
189954/189954 [==============================] - 13s 69us/step - loss: 0.0309 - acc: 0.9878 - val_loss: 0.0296 - val_acc: 0.9875
Epoch 7/100
189954/189954 [==============================] - 13s 67us/step - loss: 0.0298 - acc: 0.988

In [1]:

# plot training history
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

#show in the graph below the acurracy on both the trained and validated
#has a ever so slight increase, meaning the model could be trained longer
#to increase the accuracy of the model.It can also be seen that the model
#has not been over or under fitted as both show some comparable skill.

NameError: name 'history' is not defined

In [131]:
# Measure accuracy
predictors = model.predict(x_test)
predictors = np.argmax(predictors,axis=1)
prediction = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(prediction, predictors)
print("Accuracy:", score * 100,"%")

Accuracy: 98.94658706844815 %


In [132]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(prediction, predictors)

In [133]:
cm

array([[28419,   277],
       [ 1057, 96883]], dtype=int64)

In [134]:
#True Positive (TP) : Observation is positive, and is predicted to be positive.
#False Negative (FN) : Observation is positive, but is predicted negative.
#True Negative (TN) : Observation is negative, and is predicted to be negative.
#False Positive (FP) : Observation is negative, but is predicted positive.

TP = 28419
FP = 277
FN = 1057
TN = 96883

In [135]:
Acc = ((TP + TN)/(TP + TN + FP + FN))

print( "Accuracy:  ", Acc * 100, '%')

Accuracy:   98.94658706844815 %


In [136]:
# Precision

PPV = ((TP)/(TP+FP))

print("Precision", PPV * 100, '%')

Precision 99.03470867019793 %


In [137]:
# Recall 

TPR = ((TP)/(TP + FN))

print("Recall: ", TPR * 100, '%')

Recall:  96.41403175464785 %
